In [1]:
import factosynth
from factosynth import *

import itertools, functools
from termcolor import cprint, colored

In [2]:
# the power operator is expansive to synthesize, since it is not a standard bitvector operator.
if '^' in Operation.operators:
    del Operation.operators['^']

# Example: `Ratio`

The ground truth (inspired by the [steel plate recipe](https://wiki.factorio.com/Steel_plate) in Factorio) is:
> ```c
> Ratio(INPUT , OUTPUT) {
>     INPUT -> iron / 5 as steel -> OUTPUT
> }
> ```

In [3]:
signals = ['iron', 'steel']
def register(*args, **kwargs):
    ans = Register.fromkeys(signals,0) # makes sure that keys are ordered the same way
    ans.update(*args, **kwargs)
    return ans

({'iron': COUNT:in, 'steel': 0}, {'iron': 0, 'steel': COUNT:in/5})

In [58]:
# specification
p_in, = z3.Consts(f'COUNT:in', Int32Sort)
spec = (
    register(iron=p_in), # INPUT
    register(steel=p_in/5), # OUTPUT
)

spec

({'iron': COUNT:in, 'steel': 0}, {'iron': 0, 'steel': COUNT:in/5})

In [59]:
# sketch
combinator_sketch = ArithmeticControlBehavior(signals=signals)

print(f"first_signal  = {combinator_sketch.first_signal}")
print(f"operation = {combinator_sketch.operation.choice}")
print(f"second_signal = {combinator_sketch.second_signal}")
print(f"output_signal = {combinator_sketch.output_signal}")

first_signal  = {0: const!1474, 1: SIGNAL:iron, 2: SIGNAL:steel, 3: SIGNAL:each}
operation = {'*': A*B, '/': If(B != 0, A/B, 0), '+': A + B, '-': A - B, '%': If(B != 0, SRem(A, B), 0), '<<': A << B%32, '>>': A >> B%32, 'AND': A & B, 'OR': A | B, 'XOR': A ^ B}
second_signal = {0: const!1479, 1: SIGNAL:iron, 2: SIGNAL:steel, 3: SIGNAL:each}
output_signal = {0: SIGNAL?:iron, 1: SIGNAL?:steel, 2: SIGNAL?:each}


### example-driven

In [4]:
examples = [
    # (INPUT, OUTPUT),
    (register(iron=5 ), register(steel=1)),
    (register(iron=20), register(steel=4)),
    (register(iron=35), register(steel=7)),
]

In [6]:
# solve from examples

solver = z3.Solver()
solver.add(combinator_sketch.valid)
for example in examples:
    INPUT, OUTPUT = example
    ANS = combinator_sketch(INPUT)
    solver.add(ANS==OUTPUT)

check = solver.check()
assert check == z3.sat, f"solution is {check}, unsat_core={solver.unsat_core()}"
model = solver.model()
# print(model)

combinator_sol = combinator_sketch.eval(model)
print(combinator_sol.to_cnide())

each * -858993459 as steel


In [41]:
# verify spec

solver2 = z3.Solver()
INPUT, OUTPUT = spec
ANS = combinator_sol(INPUT)
guarantee = ANS==OUTPUT
solver2.add(z3.Not(guarantee)) # trying to find counterexamples
check = solver2.check()
if check == z3.unsat:
    cprint("Synthesized circuit complies with spec!", color='green')
elif check == z3.sat:
    model2 = solver2.model()
    cprint("Synthesized circuit does not comply with spec! Counterexample is:", color='red')
    # print(model2)
    counterexample = tuple(
        Register((signal,model2.eval(count)) for signal,count in reg.items())
        for reg in spec
    )
    INPUT, OUTPUT = counterexample
    ANS = combinator_sol(INPUT)
    print(f"""INPUT  = {INPUT.simplify(bv2int=True)}""")
    print(f"""OUTPUT = {ANS.simplify(bv2int=True)}{colored(f" ≠ {OUTPUT.simplify(bv2int=True)}", color='red')}""")

Synthesized circuit does not comply with spec! Counterexample is:
INPUT = {'iron': 1, 'steel': 0}
OUTPUT = {'iron': 0, 'steel': -858993459} ≠ {'iron': 0, 'steel': 0}


### spec + counter-example driven

In [65]:
examples = [
    # (INPUT, OUTPUT),
]

In [66]:
while True:

    # Solve from examples
    solver = z3.Solver()
    solver.add(combinator_sketch.valid)
    for example in examples:
        INPUT, OUTPUT = example
        ANS = combinator_sketch(INPUT)
        solver.add(ANS==OUTPUT)

    check = solver.check()
    assert check == z3.sat, f"solution is {check}, unsat_core={solver.unsat_core()}"
    model = solver.model()
    # print(model)

    combinator_sol = combinator_sketch.eval(model)
    print(combinator_sol.to_cnide())

    # generate counter-example
    solver2 = z3.Solver()
    INPUT, OUTPUT = spec
    ANS = combinator_sol(INPUT)
    guarantee = ANS==OUTPUT
    solver2.add(z3.Not(guarantee)) # trying to find counterexamples
    check = solver2.check()
    if check == z3.unsat:
        cprint("No more counterexamples!", color='green')
        break
    elif check == z3.sat:
        model2 = solver2.model()
        # print(model2)
        counterexample = tuple(
            Register((signal,model2.eval(count)) for signal,count in reg.items())
            for reg in spec
        )
        examples.append(counterexample)
        cprint(f"{tuple(reg.simplify(bv2int=True) for reg in counterexample)}", color='yellow')


const!1474 % each as iron
({'iron': 351273181, 'steel': 0}, {'iron': 0, 'steel': 70254636})
70254636 >> -698613248 as steel
({'iron': -1342177282, 'steel': 0}, {'iron': 0, 'steel': -268435456})
each / 5 as steel
No more counterexamples!


# Example: `Whitelist` (holes)

The ground truth is:
> ```c
> Whitelist(INPUT,MASK,OUTPUT) {
>     // set the filter to -2^31 for **each** signal on MASK
>     MASK -> 0 - each as each -> MASK_1B
>     MASK -> each + -2147483648 as each -> MASK_1B
>     
>     // filter the 31 least significant bits
>     INPUT -> each & 2147483647 as each -> INPUT_31L
>     (INPUT_31L,MASK_1B) -> each < 0 then each -> OUTPUT
>     
>     // correct the 1 most significant bit
>     INPUT -> each >> 31 as each -> INPUT_1B
>     (INPUT_1B,MASK_1B) -> each = -2147483648 then each -> OUTPUT
> }
> ```

We assume here that the combinator behaviors are unknown. This is roughly:
> ```c
> Whitelist(INPUT,MASK,OUTPUT) {
>     MASK                 ->  ?left ?op ?second as ?output     ->  MASK_1B
>     MASK                 ->  ?left ?op ?second as ?output     ->  MASK_1B
>     INPUT                ->  ?left ?op ?second as ?output     ->  INPUT_31L
>     INPUT                ->  ?left ?op ?second as ?output     ->  INPUT_1B
>     (INPUT_31L,MASK_1B)  ->  ?left ?cmp ?second then ?output  ->  OUTPUT
>     (INPUT_1B,MASK_1B)   ->  ?left ?cmp ?second then ?output  ->  OUTPUT
> }
> ```

In [74]:
# estimates the number of programs
prod = lambda values: functools.reduce(lambda a,b: a*b, values, 1)

arithmetic_hole = ArithmeticControlBehavior(signals=[])
arithmetic_combinations = prod([
    len(arithmetic_hole.first_signal),
    len(arithmetic_hole.operation.choice),
    len(arithmetic_hole.second_signal),
    len(arithmetic_hole.output_signal),
])

decider_hole = DeciderControlBehavior(signals=[])
decider_combinations = prod([
    len(decider_hole.first_signal),
    len(decider_hole.comparator.choice),
    len(decider_hole.second_signal),
    len(decider_hole.output_signal),
    2, # decider_hole.copy_count_from_input is a z3.BoolRef
])

cprint("Upper bound of possible combinator behaviors (to numeric constant):", color='blue')
print(f"Arithmetic combinator: {arithmetic_combinations}")
print(f"Decider combinator:    {decider_combinations}")
print(f"Whitelist sketch:      {arithmetic_combinations**4 * decider_combinations**2:.2e}")

Upper bound of possible combinator behaviors (to numeric constant):
Arithmetic combinator: 40
Decider combinator:    108
Whitelist sketch:      2.99e+10


In [29]:
signals = [] # we don't want to use any specific signal in the sketch
# def register(*args, **kwargs):
#     ans = Register.fromkeys(signals,0) # makes sure that keys are ordered the same way
#     ans.update(*args, **kwargs)
#     return ans

In [4]:
# specification
p_in, p_mask = params = z3.Consts(f'COUNT:in COUNT:mask', Int32Sort)
p1, = z3.Consts(f'COUNT:p1', Int32Sort)
spec = (
    Register(X=p_in),                       # INPUT
    Register(X=p_mask),                     # MASK
    Register(X=z3.If(p_mask!=0, p_in, 0)),  # OUTPUT
)

In [52]:
# sketch

cnide_sketch = """
Whitelist(INPUT,MASK,OUTPUT) {{
    MASK -> {} -> MASK_1B
    MASK -> {} -> MASK_1B
    INPUT -> {} -> INPUT_31L
    INPUT -> {} -> INPUT_1B
    (INPUT_31L,MASK_1B) -> {} -> OUTPUT
    (INPUT_1B,MASK_1B) -> {} -> OUTPUT
}}
""".strip('\n')

def circuit(INPUT, MASK, *, solver=None, combinators):
    MASK_1B = Register.sum(
        combinators[0](MASK, solver=solver),
        combinators[1](MASK, solver=solver),
    )
    # if solver is not None:
    #     MASK_1B, ans = Register.fromkeys(MASK_1B), MASK_1B
    #     solver.add(MASK_1B==ans)
    INPUT_31L = Register.sum(
        combinators[2](INPUT, solver=solver),
    )
    # if solver is not None:
    #     INPUT_31L, ans = Register.fromkeys(INPUT_31L), INPUT_31L
    #     solver.add(INPUT_31L==ans)
    INPUT_1B = Register.sum(
        combinators[3](INPUT, solver=solver),
    )
    # if solver is not None:
    #     INPUT_1B, ans = Register.fromkeys(INPUT_1B), INPUT_1B
    #     solver.add(INPUT_1B==ans)
    OUTPUT = Register.sum(
        combinators[4](INPUT_31L+MASK_1B, solver=solver),
        combinators[5](INPUT_1B+MASK_1B, solver=solver),
    )
    return OUTPUT

# combinators_sketch = [ # solution
#     ArithmeticControlBehavior(0,'-',EACH,EACH),
#     ArithmeticControlBehavior(EACH,'+',-2**31,EACH),
#     ArithmeticControlBehavior(EACH,'AND',2**31-1,EACH),
#     ArithmeticControlBehavior(EACH,'>>',31,EACH),
#     DeciderControlBehavior(EACH,'<',0,EACH,True),
#     DeciderControlBehavior(EACH,'=',-2**31,EACH,True),
# ]
# combinators_sketch = [ # full sketch (wire connections are still known)
#     ArithmeticControlBehavior(signals=[]),
#     ArithmeticControlBehavior(signals=[]),
#     ArithmeticControlBehavior(signals=[]),
#     ArithmeticControlBehavior(signals=[]),
#     DeciderControlBehavior(signals=[]),
#     DeciderControlBehavior(signals=[]),
# ]
# combinators_sketch = [ # consts missing
#     ArithmeticControlBehavior(z3.FreshConst(Int32Sort),'-',EACH,EACH),
#     ArithmeticControlBehavior(EACH,'+',z3.FreshConst(Int32Sort),EACH),
#     ArithmeticControlBehavior(EACH,'AND',z3.FreshConst(Int32Sort),EACH),
#     ArithmeticControlBehavior(EACH,'>>',z3.FreshConst(Int32Sort),EACH),
#     DeciderControlBehavior(EACH,'<',z3.FreshConst(Int32Sort),EACH,True),
#     DeciderControlBehavior(EACH,'=',z3.FreshConst(Int32Sort),EACH,True),
# ]
combinators_sketch = [ # solution with one hole
    ArithmeticControlBehavior(0,'-',EACH,EACH),
    ArithmeticControlBehavior(EACH,'+',-2**31,EACH),
    ArithmeticControlBehavior(EACH,signals=[]), # hole (BUG: couldn't remove first_signal)
    ArithmeticControlBehavior(EACH,'>>',31,EACH),
    DeciderControlBehavior(EACH,'<',0,EACH,True),
    DeciderControlBehavior(EACH,'=',-2**31,EACH,True),
]

In [53]:
examples = [
    # (INPUT, MASK, OUTPUT),
    (Register(A=50,B=40,C=30,D=-10,E=-20), Register(A=1,C=7,E=-3), Register(A=50,C=30,E=-20)),
]

### example-driven

In [54]:
# solve from examples

solver = z3.Solver()
solver.set('threads', 4)
solver.set(unsat_core=True)
for combinator in combinators_sketch: solver.add(combinator.valid)
for example in examples:
    INPUT, MASK, OUTPUT = example
    ANS = circuit(INPUT, MASK, combinators=combinators_sketch,
        solver=solver,
    )
    solver.add(ANS==OUTPUT)

check = solver.check()
assert check == z3.sat, f"solution is {check}, unsat_core={solver.unsat_core()}"
model = solver.model()
# print(model)

combinators_sol = [combinator.eval(model) for combinator in combinators_sketch]
print(cnide_sketch.format(*[combinator.to_cnide() for combinator in combinators_sol]))

Whitelist(INPUT,MASK,OUTPUT) {
    MASK -> 0 - each as each -> MASK_1B
    MASK -> each + -2147483648 as each -> MASK_1B
    INPUT -> each AND 2147483646 as each -> INPUT_31L
    INPUT -> each >> 31 as each -> INPUT_1B
    (INPUT_31L,MASK_1B) -> each < 0 then each -> OUTPUT
    (INPUT_1B,MASK_1B) -> each = -2147483648 then each -> OUTPUT
}


In [65]:
# verify spec

solver2 = z3.Solver()
INPUT, MASK, OUTPUT = spec
ANS = circuit(INPUT, MASK, combinators=combinators_sol)
guarantee = ANS==OUTPUT
solver2.add(z3.Not(guarantee)) # trying to find counterexamples
check = solver2.check()
if check == z3.unsat:
    cprint("Synthesized circuit complies with spec!", color='green')
elif check == z3.sat:
    model2 = solver2.model()
    cprint("Synthesized circuit does not comply with spec! Counterexample is:", color='red')
    # print(model2)
    counterexample = tuple(
        Register((signal,model2.eval(count)) for signal,count in reg.items())
        for reg in spec
    )
    INPUT, MASK, OUTPUT = counterexample
    ANS = circuit(INPUT, MASK, combinators=combinators_sol)
    print(f"""INPUT  = {INPUT.simplify(bv2int=True)}""")
    print(f"""MASK   = {MASK.simplify(bv2int=True)}""")
    print(f"""OUTPUT = {ANS.simplify(bv2int=True)}{colored(f" ≠ {OUTPUT.simplify(bv2int=True)}", color='red')}""")

Synthesized circuit does not comply with spec! Counterexample is:
INPUT  = {'X': -1}
MASK   = {'X': 6}
OUTPUT = {'X': -2} ≠ {'X': -1}


### spec + counter-example guided

In [66]:
examples = [
    # (INPUT, MASK, OUTPUT),
]

In [67]:
while True:

    # solve from examples
    solver = z3.Solver()
    solver.set('threads', 4)
    solver.set(unsat_core=True)
    for combinator in combinators_sketch: solver.add(combinator.valid)
    for example in examples:
        INPUT, MASK, OUTPUT = example
        ANS = circuit(INPUT, MASK, combinators=combinators_sketch,
            solver=solver,
        )
        solver.add(ANS==OUTPUT)

    check = solver.check()
    assert check == z3.sat, f"solution is {check}, unsat_core={solver.unsat_core()}"
    model = solver.model()
    # print(model)

    combinators_sol = [combinator.eval(model) for combinator in combinators_sketch]
    print(cnide_sketch.format(*[combinator.to_cnide() for combinator in combinators_sol]))

    # generate counter-example
    solver2 = z3.Solver()
    INPUT, MASK, OUTPUT = spec
    ANS = circuit(INPUT, MASK, combinators=combinators_sol,
        # solver=solver,
    )
    guarantee = ANS==OUTPUT
    solver2.add(z3.Not(guarantee)) # trying to find counterexamples
    check = solver2.check()
    if check == z3.unsat:
        cprint("No more counterexamples!", color='green')
        break
    elif check == z3.sat:
        model2 = solver2.model()
        # print(model2)
        counterexample = tuple(
            Register((signal,model2.eval(count)) for signal,count in reg.items())
            for reg in spec
        )
        examples.append(counterexample)
        INPUT, MASK, OUTPUT = counterexample
        cprint(f"""INPUT  = {INPUT.simplify(bv2int=True)}""", color='yellow')
        cprint(f"""MASK   = {MASK.simplify(bv2int=True)}""", color='yellow')
        cprint(f"""OUTPUT = {OUTPUT.simplify(bv2int=True)}""", color='yellow')

    # solver2 = z3.Solver()
    # INPUT, OUTPUT = spec
    # ANS = combinator_sol(INPUT)
    # guarantee = ANS==OUTPUT
    # solver2.add(z3.Not(guarantee)) # trying to find counterexamples
    # check = solver2.check()
    # if check == z3.unsat:
    #     cprint("No more counterexamples!", color='green')
    #     break
    # elif check == z3.sat:
    #     model2 = solver2.model()
    #     # print(model2)
    #     counterexample = tuple(
    #         Register((signal,model2.eval(count)) for signal,count in reg.items())
    #         for reg in spec
    #     )
    #     examples.append(counterexample)
    #     cprint(f"{tuple(reg.simplify(bv2int=True) for reg in counterexample)}", color='yellow')

Whitelist(INPUT,MASK,OUTPUT) {
    MASK -> 0 - each as each -> MASK_1B
    MASK -> each + -2147483648 as each -> MASK_1B
    INPUT -> each / const!4330 as each -> INPUT_31L
    INPUT -> each >> 31 as each -> INPUT_1B
    (INPUT_31L,MASK_1B) -> each < 0 then each -> OUTPUT
    (INPUT_1B,MASK_1B) -> each = -2147483648 then each -> OUTPUT
}
INPUT  = {'X': 1109652381}
MASK   = {'X': 0}
OUTPUT = {'X': 0}
Whitelist(INPUT,MASK,OUTPUT) {
    MASK -> 0 - each as each -> MASK_1B
    MASK -> each + -2147483648 as each -> MASK_1B
    INPUT -> each XOR 35910557 as each -> INPUT_31L
    INPUT -> each >> 31 as each -> INPUT_1B
    (INPUT_31L,MASK_1B) -> each < 0 then each -> OUTPUT
    (INPUT_1B,MASK_1B) -> each = -2147483648 then each -> OUTPUT
}
INPUT  = {'X': 4}
MASK   = {'X': 1073741824}
OUTPUT = {'X': 4}
Whitelist(INPUT,MASK,OUTPUT) {
    MASK -> 0 - each as each -> MASK_1B
    MASK -> each + -2147483648 as each -> MASK_1B
    INPUT -> each XOR 0 as each -> INPUT_31L
    INPUT -> each >> 31 as e

## Example: `Whitelist` (connections)

The ground truth is:
> ```c
> Whitelist(INPUT,MASK,OUTPUT) {
>     // set the filter to -2^31 for **each** signal on MASK
>     MASK -> 0 - each as each -> MASK_1B
>     MASK -> each + -2147483648 as each -> MASK_1B
>     
>     // filter the 31 least significant bits
>     INPUT -> each & 2147483647 as each -> INPUT_31L
>     (INPUT_31L,MASK_1B) -> each < 0 then each -> OUTPUT
>     
>     // correct the 1 most significant bit
>     INPUT -> each >> 31 as each -> INPUT_1B
>     (INPUT_1B,MASK_1B) -> each = -2147483648 then each -> OUTPUT
> }
> ```

We assume here that the wire connections are unknown. This is roughly:
> ```c
> Whitelist(INPUT,MASK,OUTPUT) {
>     (??,??)  ->  0 - each as each             ->  (??,??)
>     (??,??)  ->  each + -2147483648 as each   ->  (??,??) 
>     (??,??)  ->  each & 2147483647 as each    ->  (??,??) 
>     (??,??)  ->  each >> 31 as each           ->  (??,??) 
>     (??,??)  ->  each < 0 then each           ->  (??,??) 
>     (??,??)  ->  each = -2147483648 then each ->  (??,??) 
> }
> ```

In [ ]:
# specification
p_in, p_mask = params = z3.Consts(f'COUNT:in, COUNT:mask', Int32Sort)
p1, = z3.Consts(f'COUNT:p1', Int32Sort)
spec = (
    Register(X=p_in),                       # INPUT
    Register(X=p_mask),                     # MASK
    Register(X=z3.If(p_mask!=0, p_in, 0)),  # OUTPUT
)

In [91]:
# sketch

combinators_sketch = [ # solution
    ArithmeticControlBehavior(0,'-',EACH,EACH),
    ArithmeticControlBehavior(EACH,'+',-2**31,EACH),
    ArithmeticControlBehavior(EACH,'AND',2**31-1,EACH),
    ArithmeticControlBehavior(EACH,'>>',31,EACH),
    DeciderControlBehavior(EACH,'<',0,EACH,True),
    DeciderControlBehavior(EACH,'=',-2**31,EACH,True),
]
wires = [
    'INPUT',
    'MASK',
    'OUTPUT',
    'INTERNAL1',
    'INTERNAL2',
    'INTERNAL3',
]

# wire_color_sketch = { # wire_color_sketch[wire]       # TODO: consider wire color as well
#     wire: Choice(red=1, green=2)
#     for wire in wires
# }
connect_sketch = { # connect[wire][combinator][connection_point]
    wire: [
        {
            1: z3.FreshConst(z3.BoolSort(), f'{wire}->combinators[{c}]'), # input
            2: z3.FreshConst(z3.BoolSort(), f'combinators[{c}]->{wire}'), # output
        }
        for c,combinator in enumerate(combinators_sketch)
    ]
    for wire in wires
}

def compute_tick(
    reg_wires:'list[Register]',
    combinators:'list[ControlBehavior]',
    connect:'dict[str,list[dict[int,z3.BoolRef]]]',
    *,
    solver=None,
):
    """compute one tick of the circuit"""
    reg_wires[0] & connect_sketch['INPUT'][2][1]
    reg_combinators_input = [
        Register.sum(*[
            reg_wire & connect_sketch[wire][c][1]
            for wire, reg_wire in zip(wires, reg_wires)
        ])
        for c,combinator in enumerate(combinators)
    ]
    if solver is not None:
        for c,ans in enumerate(reg_combinators_input):
            out = Register.fromkeys(ans)
            solver.add(out==ans)
            reg_combinators_input[c] = out
    
    reg_combinators_output = [
        combinator(input, solver=solver)
        for combinator, input in zip(combinators, reg_combinators_input)
    ]
    
    reg_wires_next = [
        Register.sum(*[
            reg_combinator_output & connect_sketch[wire][c][2]
            for c,reg_combinator_output in enumerate(reg_combinators_output)
        ])
        for wire in wires
    ]
    if solver is not None:
        for c,ans in enumerate(reg_wires_next):
            out = Register.fromkeys(ans)
            solver.add(out==ans)
            reg_wires_next[c] = out
    
    return reg_wires_next
def circuit(INPUT, MASK, delay=2, *, combinators, connect, solver=None):
    reg_wires = [
        INPUT  if wire=='INPUT' else
        MASK   if wire=='MASK' else
        Register()
        for wire in wires
    ]
    for tick in range(delay):
        reg_wires = compute_tick(reg_wires, combinators, connect, solver=solver)
    OUTPUT = reg_wires[wires.index('OUTPUT')]
    return OUTPUT

In [78]:
cprint("Upper bound of possible combinator behaviors (to numeric constant):", color='blue')
print(f"point connections: {2**( len(wires) * len(combinators_sketch) * len([1,2]) ):.2e}")

Upper bound of possible combinator behaviors (to numeric constant):
point connections: 4.72e+21


In [80]:
examples = [
    # (INPUT, MASK, OUTPUT),
    (Register(A=50,B=40,C=30,D=-10,E=-20), Register(A=1,C=7,E=-3), Register(A=50,C=30,E=-20)),
]

### example-driven

In [92]:
# solve from examples

solver = z3.Solver()
solver.set('threads', 4)
solver.set(unsat_core=True)
for combinator in combinators_sketch: solver.add(combinator.valid)
for example in examples:
    INPUT, MASK, OUTPUT = example
    ANS = circuit(INPUT, MASK, combinators=combinators_sketch, connect=connect_sketch,
        solver=solver,
    )
    solver.add(ANS==OUTPUT)

check = solver.check()
assert check == z3.sat, f"solution is {check}, unsat_core={solver.unsat_core()}"
model = solver.model()
# print(model)

combinators_sol = [combinator.eval(model) for combinator in combinators_sketch]
connect_sol = { # connect[wire][combinator][connection_point]
    wire: [
        {
            cp: model.eval(connect_wire_c_cp)
            for cp,connect_wire_c_cp in connect_wire_c.items()
        }
        for connect_wire_c in connect_wire
    ]
    for wire,connect_wire in connect_sketch.items()
}

# print solved sketch
for c,combinator in enumerate(combinators_sketch):
    inputs = tuple(
        wire
        for wire in wires
        if connect_sol[wire][c][1]
    )
    outputs = tuple(
        wire
        for wire in wires
        if connect_sol[wire][c][1]
    )
    if len(inputs)==1: inputs = inputs[0]
    if len(outputs)==1: outputs = outputs[0]
    print(f"{inputs} -> {combinator.to_cnide()} -> {outputs}")

: 

: 